<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>The Battle of Neighborhoods (Week 2): Toronto vs. Frankfurt
</font></h1>

### Week 5 Assignment
Author: Matthias Reumann


## Introduction

This Jupyter file is my submission of the week 5 assignment "The battle of neighborhoods (week 2)". 

This project focusses on the challenge of comparing two cities with respect to standard of Living (SoL). The interest in the analysis and results can come from multiple stakeholders on individual but also on corporate level: 
1.	As an individual relocating to different cities, one wants to understand which city offers more and has what kind of character that fits ones personals preferences.
2.	Big corporations might be interested in this kind of analysis if they have to relocate their operations to another town.

The work is motivated two folds. One one hand, the authors personal journey moving from Frankfurt/Main, Germany to Southampton, UK with a brief stint in London, UK, then Paris, France, then finally New York, USA, Melbourne, Australia and Zürich, Switzerland has given him a deep insight into the cultures and feels of a city and an understanding why it is important to have data when moving across continents. On the other, the recent changes in the financial market due to the impending Brexit has many banks move from London to a different location (https://news.efinancialcareers.com/uk-en/3000398/brexit-banks-moving-out-of-london). An important part in deciding where to move is whether the infrastructure or more generally the SoL is the best fit for the person or institution making that choice.

An in-depth analysis of comparing two or more locations is beyond the scope of this work. Hence, the focus is narrowed down to evaluate and compare two cities with respect to food diversity and quality in the neighbourhoods of Toronto and Frankfurt/Main as a measure of SoL of the two cities.


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset - data wrangling with wikipedia page</a>

2. <a href="#item2">Explore Neighborhoods in Toronto and Frankfurt</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import sys
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge lxml --yes # Uncomment if lxml is not installed
# Install a conda package in the current Jupyter kernel
#import sys
!conda install --yes --prefix {sys.prefix} lxml

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         238 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

<a id='item1'></a>

## 1. Download and Explore Datasets

## Toronto Data

Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.
- The rows with Boroughs that are not assigned are dropped. 
- The neighborhoods are grouped by Boroughs
- The geolocations are added using the Geospatial_Coordinates.csv file


In [2]:

url_Toronto = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tables = pd.read_html(url_Toronto, header=0)
#print(tables[0].columns.values)
dfToronto = tables[0]


# drop 'not assigned' fields in 'Borough'
# Get names of indexes
index = dfToronto[dfToronto['Borough'] == 'Not assigned'].index
# Delete these row indexes from dataFrame
dfToronto.drop(index, inplace=True)

# assign Neighbourhood 'Not assigned' to Borough name
index = dfToronto[dfToronto['Neighbourhood'] == 'Not assigned'].index

for i in index:
    dfToronto.loc[i]['Neighbourhood'] = dfToronto.loc[i]['Borough']

# groupby Boroughs keeping the Neighbourhood as lists
#dfToronto = dfToronto.groupby(['Borough']).agg({'Postcode': list, 'Neighbourhood': list}).reset_index()
dfToronto = dfToronto.groupby(['Borough', 'Postcode']).agg({'Neighbourhood': list}).reset_index()

#print(dfToronto.columns.values)
dfToronto = dfToronto[['Postcode', 'Borough', 'Neighbourhood']]


# add geolocation data
c=pd.read_csv("Geospatial_Coordinates.csv")

#print(c.head())

dfToronto['Latitude'] = 0.0
dfToronto['Longitude'] = 0.0

for index, row in dfToronto.iterrows():
    dfToronto.at[index, 'Latitude'] = c.loc[c['Postal Code'] == row['Postcode'], 'Latitude']
    dfToronto.at[index, 'Longitude'] = c.loc[c['Postal Code'] == row['Postcode'], 'Longitude']




In [3]:
print(dfToronto.columns.values)
print(dfToronto.shape)
dfToronto[dfToronto['Borough'] == 'Central Toronto'].head()
dfToronto_Central = dfToronto[dfToronto['Borough'] == 'Central Toronto']
dfToronto_Central

['Postcode' 'Borough' 'Neighbourhood' 'Latitude' 'Longitude']
(103, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790
1,M4P,Central Toronto,[Davisville North],43.712751,-79.390197
2,M4R,Central Toronto,[North Toronto West],43.715383,-79.405678
3,M4S,Central Toronto,[Davisville],43.704324,-79.388790
4,M4T,Central Toronto,"[Moore Park, Summerhill East]",43.689574,-79.383160
5,M4V,Central Toronto,"[Deer Park, Forest Hill SE, Rathnelly, South H...",43.686412,-79.400049
6,M5N,Central Toronto,[Roselawn],43.711695,-79.416936
7,M5P,Central Toronto,"[Forest Hill North, Forest Hill West]",43.696948,-79.411307
8,M5R,Central Toronto,"[The Annex, North Midtown, Yorkville]",43.672710,-79.405678


## Frankfurt Data

In [4]:

url_FFM = 'https://en.wikipedia.org/wiki/List_of_Ortsbezirke_of_Frankfurt_am_Main'

tables = pd.read_html(url_FFM, header=0)
print(tables[0].columns.values)

dfFFM = tables[0]

print(dfFFM.columns.values)
print(dfFFM.shape)
dfFFM.head()

['No.' 'Name' 'City districts' 'Population' 'Area(km²)'
 'Population density(Population/km²)']
['No.' 'Name' 'City districts' 'Population' 'Area(km²)'
 'Population density(Population/km²)']
(16, 6)


,No.,Name,City districts,Population,Area(km²),Population density(Population/km²)
0,1,Innenstadt I,"Altstadt, Bahnhofsviertel, Gallus, Gutleutvier...",44.183,8987,4.916
1,2,Innenstadt II,"Bockenheim, Westend-Nord, Westend-Süd,",57.629,9493,6.071
2,3,Innenstadt III,"Nordend-Ost, Nordend-West",51.671,4744,10.892
3,4,Bornheim/Ostend,"Ostend, Bornheim",56.723,9212,6.158
4,5,Süd,"Flughafen, Niederrad, Oberrad, Sachsenhausen-S...",91.662,84831,1.081


In [5]:
dfFFM = dfFFM[['Name', 'City districts']]

print(dfFFM.columns.values)
print(dfFFM.shape)
dfFFM.head()

['Name' 'City districts']
(16, 2)


,Name,City districts
0,Innenstadt I,"Altstadt, Bahnhofsviertel, Gallus, Gutleutvier..."
1,Innenstadt II,"Bockenheim, Westend-Nord, Westend-Süd,"
2,Innenstadt III,"Nordend-Ost, Nordend-West"
3,Bornheim/Ostend,"Ostend, Bornheim"
4,Süd,"Flughafen, Niederrad, Oberrad, Sachsenhausen-S..."


In [6]:
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Altstadt_(Frankfurt_am_Main)'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Bahnhofsviertel'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Gallus_(Frankfurt_am_Main)'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Gutleutviertel'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Bockenheim_(Frankfurt_am_Main)'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Innenstadt_(Frankfurt_am_Main)'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Westend_(Frankfurt_am_Main)'
#url_FFM_neighb = 'https://en.wikipedia.org/wiki/Nordend_(Frankfurt_am_Main)'

#tables = pd.read_html(url_FFM_neighb, header=0)
#url_FFM_neighb = tables[0]
#url_FFM_neighb.iloc[4, 0]

In [7]:
dfFFM_Central = pd.read_csv("CentralFrankfurt.csv", sep = ';')

dfFFM_Central.columns.values
dfFFM_Central


,Borough,Neighbourhood,Latitude,Longitude,ZIP code
0,Innenstadt I,Altstadt,50.113610,8.684444,60311; 60313
1,Innenstadt I,Bahnhofsviertel,50.101667,8.659444,60329
2,Innenstadt I,Gallus,50.103611,8.645556,60325; 60326; 60327; 60329; 60486
3,Innenstadt I,Gutleutviertel,50.101667,8.659444,60327; 60329
4,Innenstadt I,Innenstadt,50.120278,8.682778,60310; 60311; 60313; 60318; 60322; 60329
5,Innenstadt III,Bockenheim,50.123889,8.639167,60325; 60431; 60486; 60487
6,Innenstadt III,Westend,50.118056,8.663333,60320; 60322; 60323; 60431; 60306; 60308; 6032...
7,Innenstadt III,Nordend,50.123056,8.692222,60316; 60318; 60385; 60389; 60316; 60318; 6032...


## 2. Explore Datasets

In [8]:
address = 'Central Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6449033, -79.3818364.


In [9]:
# create map of Toronto using latitude and longitude values

map_CentralToronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(dfToronto_Central['Latitude'], dfToronto_Central['Longitude'], dfToronto_Central['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CentralToronto)  
    
map_CentralToronto

In [10]:
address = 'Central Frankfurt, Hesse'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Frankfurt are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Frankfurt are 50.117794, 8.6864491.


In [11]:
# create map of Manhattan using latitude and longitude values

map_CentralFrankfurt = folium.Map(location=[latitude, longitude], zoom_start=11)



# add markers to map
for lat, lng, label in zip(dfFFM_Central['Latitude'], dfFFM_Central['Longitude'], dfFFM_Central['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_CentralFrankfurt)  


map_CentralFrankfurt

## 3. Analyze each neighborhood

### Adding Foursquare Credentials
NOTE: add your own credentials here!!!

In [12]:
#CLIENT_ID = 'your-client-ID' # your Foursquare ID
#CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
CLIENT_ID = '04VNSYGDD2TVCPIAY30LOCDB4VU2QR3QMVL4BQIEIR240323' # your Foursquare ID
CLIENT_SECRET = 'JEGIPA2GT501NS0AH4SRCTZFW2LFGV3JK3JNKCJZE4HVZFKB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 04VNSYGDD2TVCPIAY30LOCDB4VU2QR3QMVL4BQIEIR240323
CLIENT_SECRET:JEGIPA2GT501NS0AH4SRCTZFW2LFGV3JK3JNKCJZE4HVZFKB


### Venues in Toronto

In [13]:
dfToronto_Central.loc[0, 'Neighbourhood']
neighborhood_latitude = dfToronto_Central.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfToronto_Central.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfToronto_Central.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Lawrence Park'] are 43.7280205, -79.3887901.


In [23]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?client_id=04VNSYGDD2TVCPIAY30LOCDB4VU2QR3QMVL4BQIEIR240323&client_secret=JEGIPA2GT501NS0AH4SRCTZFW2LFGV3JK3JNKCJZE4HVZFKB&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [24]:
# send GET request and get trending venues
results = requests.get(url).json()
#results
'There are {} venues around Central Toronto (because I limited the request).'.format(len(results['response']['groups'][0]['items']))

'There are 3 venues around Central Toronto (because I limited the request).'

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories
0,Lawrence Park Ravine,Park
1,Zodiac Swim School,Swim School
2,TTC Bus #162 - Lawrence-Donway,Bus Line


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods

In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
dfToronto_venues = getNearbyVenues(names=dfToronto_Central['Neighbourhood'],
                                   latitudes=dfToronto_Central['Latitude'],
                                   longitudes=dfToronto_Central['Longitude']
                                  )


dfToronto_venues['Neighborhood'] = dfToronto_venues['Neighborhood'].apply(', '.join)

['Lawrence Park']
['Davisville North']
['North Toronto West']
['Davisville']
['Moore Park', 'Summerhill East']
['Deer Park', 'Forest Hill SE', 'Rathnelly', 'South Hill', 'Summerhill West']
['Roselawn']
['Forest Hill North', 'Forest Hill West']
['The Annex', 'North Midtown', 'Yorkville']


In [65]:
print(dfToronto_venues.shape)
dfToronto_venues.head()

(113, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


#### Let's find out how many unique categories can be curated from all the returned venues

In [66]:
dfToronto_venues.groupby(['Neighborhood']).count()
print('There are {} uniques categories.'.format(len(dfToronto_venues['Venue Category'].unique())))

There are 61 uniques categories.


#### Analyze Each Neighborhood

In [69]:
# one hot encoding
Toronto_onehot = pd.get_dummies(dfToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = dfToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

print(Toronto_onehot.shape)
print(Toronto_onehot.columns.values)
Toronto_onehot.head()

(113, 62)
['Neighborhood' 'American Restaurant' 'Asian Restaurant' 'BBQ Joint'
 'Bagel Shop' 'Breakfast Spot' 'Brewery' 'Burger Joint' 'Bus Line' 'Café'
 'Chinese Restaurant' 'Clothing Store' 'Coffee Shop' 'Convenience Store'
 'Cosmetics Shop' 'Dance Studio' 'Dessert Shop' 'Diner' 'Farmers Market'
 'Food & Drink Shop' 'Fried Chicken Joint' 'Garden' 'Gourmet Shop'
 'Greek Restaurant' 'Gym' 'Gym / Fitness Center' 'Health & Beauty Service'
 'History Museum' 'Home Service' 'Hotel' 'Indian Restaurant'
 'Italian Restaurant' 'Jewelry Store' 'Jewish Restaurant'
 'Light Rail Station' 'Liquor Store' 'Metro Station' 'Mexican Restaurant'
 'Movie Theater' 'Park' 'Pharmacy' 'Pizza Place' 'Playground' 'Pub'
 'Rental Car Location' 'Restaurant' 'Salon / Barbershop' 'Sandwich Place'
 'Seafood Restaurant' 'Spa' 'Sporting Goods Shop' 'Sports Bar'
 'Supermarket' 'Sushi Restaurant' 'Swim School' 'Tennis Court'
 'Thai Restaurant' 'Toy / Game Store' 'Trail'
 'Vegetarian / Vegan Restaurant' 'Vietnamese Restaur

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Metro Station,Mexican Restaurant,Movie Theater,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [73]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()

print(Toronto_grouped.shape)
Toronto_grouped

(9, 62)


,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Garden,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Health & Beauty Service,History Museum,Home Service,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Metro Station,Mexican Restaurant,Movie Theater,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.030303,0.000000,0.000000,0.0,0.030303,0.000000,0.000000,0.060606,0.000000,0.000000,0.060606,0.0,0.000000,0.0,0.090909,0.030303,0.030303,0.0,0.030303,0.0,0.030303,0.030303,0.060606,0.000000,0.0000,0.000000,0.0,0.0,0.030303,0.060606,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.030303,0.030303,0.030303,0.060606,0.00,0.000000,0.000000,0.030303,0.000000,0.090909,0.030303,0.000000,0.000000,0.0000,0.0000,0.060606,0.000000,0.00,0.030303,0.030303,0.0,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.1,0.000000,0.1,0.000000,0.000000,0.000000,0.1,0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.0000,0.000000,0.0,0.1,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.100000,0.00,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.062500,0.000000,0.000000,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0625,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.00,0.125000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.0625,0.062500,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0625,0.000000
3,"Forest Hill North, Forest Hill West",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.2,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.200000,0.000000,0.00,0.000000,0.000000,0.2,0.000000,0.0000,0.000000
4,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.333333,0.00,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
5,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
6,North Toronto West,0.000000,0.000000,

#### Let's print each neighborhood along with the top 5 most common venues

In [75]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2  Italian Restaurant  0.06
3    Sushi Restaurant  0.06
4         Coffee Shop  0.06


----Davisville North----
               venue  freq
0  Food & Drink Shop   0.1
1     Breakfast Spot   0.1
2               Park   0.1
3                Gym   0.1
4              Hotel   0.1


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                     venue  freq
0              Coffee Shop  0.12
1                      Pub  0.12
2      American Restaurant  0.06
3  Health & Beauty Service  0.06
4              Pizza Place  0.06


----Forest Hill North, Forest Hill West----
              venue  freq
0  Sushi Restaurant   0.2
1     Jewelry Store   0.2
2             Trail   0.2
3          Bus Line   0.2
4              Park   0.2


----Lawrence Park----
                 venue  freq
0             Bus Line  0.33
1                 Park  0.33
2          Swim School  0.33
3  Ameri

#### Let's put that into a *pandas* dataframe

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Italian Restaurant,Sushi Restaurant,Pizza Place,Café,Coffee Shop,Gym,Park,Greek Restaurant
1,Davisville North,Dance Studio,Clothing Store,Hotel,Food & Drink Shop,Sandwich Place,Convenience Store,Park,Pizza Place,Gym,Breakfast Spot
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,American Restaurant,Sports Bar,Pizza Place,Health & Beauty Service,Restaurant,Liquor Store,Light Rail Station,Fried Chicken Joint
3,"Forest Hill North, Forest Hill West",Sushi Restaurant,Bus Line,Trail,Jewelry Store,Park,Farmers Market,Food & Drink Shop,Fried Chicken Joint,Garden,Yoga Studio
4,Lawrence Park,Swim School,Bus Line,Park,Yoga Studio,Dessert Shop,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant
5,"Moore Park, Summerhill East",Restaurant,Gym,Tennis Court,Playground,Dance Studio,History Museum,Health & Beauty Service,Gym / Fitness Center,Greek Restaurant,Gourmet Shop
6,North Toronto West,Coffee Shop,Sporting Goods Shop,Yoga Studio,Spa,Metro Station,Mexican Restaurant,Diner,Park,Dessert Shop,Restaurant
7,Roselawn,Home Service,Garden,Yoga Studio,Indian Restaurant,History Museum,Health & Beauty Service,Gym / Fitness Center,Gym,Greek Restaurant,Gourmet Shop
8,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,American Restaurant,Indian Restaurant,Jewish Restaurant,Liquor Store,Park,Pharmacy,Pizza Place


### Venues in Frankfurt

In [79]:
dfFFM_Central.loc[0, 'Neighbourhood']
neighborhood_latitude = dfFFM_Central.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfFFM_Central.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfFFM_Central.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Altstadt are 50.113609999999994, 8.684444000000001.


In [80]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?client_id=04VNSYGDD2TVCPIAY30LOCDB4VU2QR3QMVL4BQIEIR240323&client_secret=JEGIPA2GT501NS0AH4SRCTZFW2LFGV3JK3JNKCJZE4HVZFKB&v=20180605&ll=50.113609999999994,8.684444000000001&radius=500&limit=100'

In [82]:
# send GET request and get trending venues
results = requests.get(url).json()
#results
'There are {} venues around Central Frankfurt (because I limited the request).'.format(len(results['response']['groups'][0]['items']))

'There are 100 venues around Central Frankfurt (because I limited the request).'

In [83]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories
0,Góc Phố,Vietnamese Restaurant
1,Kleinmarkthalle,Market
2,Weinterasse Rollanderhof,Wine Bar
3,FITSEVENELEVEN Zeil,Gym / Fitness Center
4,Erzeugermarkt Konstablerwache,Farmers Market


In [84]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


#### Let's create a function to repeat the same process to all the neighborhoods

In [92]:
dfFFM_venues = getNearbyVenues(names=dfFFM_Central['Neighbourhood'],
                                   latitudes=dfFFM_Central['Latitude'],
                                   longitudes=dfFFM_Central['Longitude']
                                  )


#dfFFM_venues['Neighborhood'] = dfFFM_venues['Neighborhood'].apply(', '.join)

Altstadt
Bahnhofsviertel
Gallus
Gutleutviertel
Innenstadt
Bockenheim
Westend
Nordend


In [93]:
print(dfFFM_venues.shape)
dfFFM_venues.head()

(353, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,50.11361,8.684444,Góc Phố,50.113509,8.681686,Vietnamese Restaurant
1,Altstadt,50.11361,8.684444,Kleinmarkthalle,50.112778,8.682958,Market
2,Altstadt,50.11361,8.684444,Weinterasse Rollanderhof,50.112473,8.682164,Wine Bar
3,Altstadt,50.11361,8.684444,FITSEVENELEVEN Zeil,50.114163,8.682001,Gym / Fitness Center
4,Altstadt,50.11361,8.684444,Erzeugermarkt Konstablerwache,50.114516,8.687116,Farmers Market


#### Let's find out how many unique categories can be curated from all the returned venues

In [94]:
dfFFM_venues.groupby(['Neighborhood']).count()
print('There are {} uniques categories.'.format(len(dfFFM_venues['Venue Category'].unique())))

There are 105 uniques categories.


#### Analyze Each Neighborhood

In [95]:
# one hot encoding
FFM_onehot = pd.get_dummies(dfFFM_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
FFM_onehot['Neighborhood'] = dfFFM_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [FFM_onehot.columns[-1]] + list(FFM_onehot.columns[:-1])
FFM_onehot = FFM_onehot[fixed_columns]

print(FFM_onehot.shape)
print(FFM_onehot.columns.values)
FFM_onehot.head()

(353, 106)
['Neighborhood' 'Accessories Store' 'African Restaurant' 'Art Museum'
 'Asian Restaurant' 'Athletics & Sports' 'Bakery' 'Bar' 'Beer Bar'
 'Beer Store' 'Bistro' 'Boarding House' 'Bookstore' 'Breakfast Spot'
 'Brewery' 'Burger Joint' 'Burrito Place' 'Butcher' 'Café'
 'Chinese Restaurant' 'Chocolate Shop' 'Clothing Store' 'Cocktail Bar'
 'Coffee Shop' 'Concert Hall' 'Cosmetics Shop' 'Currywurst Joint'
 'Department Store' 'Dessert Shop' 'Dim Sum Restaurant' 'Donut Shop'
 'Drugstore' 'Eastern European Restaurant' 'Electronics Store'
 'Event Space' 'Falafel Restaurant' 'Farmers Market'
 'Fast Food Restaurant' 'Food & Drink Shop' 'Food Court'
 'French Restaurant' 'Garden' 'Gay Bar' 'German Restaurant' 'Gourmet Shop'
 'Greek Restaurant' 'Grocery Store' 'Gym' 'Gym / Fitness Center'
 'Historic Site' 'History Museum' 'Hookah Bar' 'Hotel' 'Hotel Bar'
 'Ice Cream Shop' 'Indian Restaurant' 'Indie Movie Theater'
 'Italian Restaurant' 'Japanese Restaurant' 'Kebab Restaurant'
 'Korean Restau

,Neighborhood,Accessories Store,African Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Bakery,Bar,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Dim Sum Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Garden,Gay Bar,German Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Museum,Nightclub,Optical Shop,Park,Pedestrian Plaza,Persian Restaurant,Peruvian Restaurant,Pizza Place,Plaza,Pub,Radio Station,Restaurant,River,Road,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Altstadt,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [96]:
FFM_grouped = FFM_onehot.groupby('Neighborhood').mean().reset_index()

print(FFM_grouped.shape)
FFM_grouped

(8, 106)


,Neighborhood,Accessories Store,African Restaurant,Art Museum,Asian Restaurant,Athletics & Sports,Bakery,Bar,Beer Bar,Beer Store,Bistro,Boarding House,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Cosmetics Shop,Currywurst Joint,Department Store,Dessert Shop,Dim Sum Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Garden,Gay Bar,German Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Korean Restaurant,Lounge,Market,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Monument / Landmark,Museum,Nightclub,Optical Shop,Park,Pedestrian Plaza,Persian Restaurant,Peruvian Restaurant,Pizza Place,Plaza,Pub,Radio Station,Restaurant,River,Road,Roof Deck,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soup Place,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Altstadt,0.010000,0.000000,0.02,0.000000,0.010000,0.000000,0.040000,0.01,0.01,0.000000,0.000000,0.01,0.010000,0.010000,0.030000,0.000,0.01,0.060000,0.000000,0.01,0.01,0.000000,0.030000,0.000000,0.01,0.010000,0.040000,0.01,0.010000,0.01,0.010000,0.01,0.01,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.030000,0.01,0.010000,0.000000,0.000,0.010000,0.01,0.01,0.000000,0.020000,0.01,0.010000,0.000000,0.010000,0.050000,0.010000,0.000000,0.000000,0.010000,0.02,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.040000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000,0.000000,0.02,0.010000,0.010000,0.01,0.01,0.000000,0.01,0.000000,0.0,0.000000,0.020000,0.010000,0.010000,0.040000,0.01,0.000000,0.010000,0.010000,0.030000,0.010000
1,Bahnhofsviertel,0.000000,0.043478,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.086957,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.043478,0.00,0.00,0.043478,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.043478,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.00,0.000000,0.173913,0.00,0.000000,0.000000,0.000000,0.086957,0.000000,0.000000,0.043478,0.000000,0.00,0.000000,0.000000,0.043478,0.043478,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.043478,0.000000,0.000000,0.000,0.043478,0.00,0.043478,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.0,0.043478,0.000000,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.000000,0.000000
2,Bockenheim,0.000000,0.000000,0.00,0.115385,0.000000,0.038462,0.038462,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.115385,0.000000,0.00,0.00,0.038462,0.000000,0.000000,0.00,0.000000,0.038462,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.038462,0.000000,0.000,0.000000,0.00,0.00,0.038462,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.00,0.038462,0.000000,0.00,0.000000,0.000000,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.00,0.038462,0.000000,0.00,0.00,0.076923,0.00,0.000000,0.0,0.076923,0.000000,0.038462,0.000000,0.000000,0.00,0.000000,0.038462,0.000000,0.038462

#### Let's print each neighborhood along with the top 5 most common venues

In [98]:
num_top_venues = 5

for hood in FFM_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = FFM_grouped[FFM_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altstadt----
                venue  freq
0                Café  0.06
1          Restaurant  0.05
2  Italian Restaurant  0.05
3               Plaza  0.04
4                 Bar  0.04


----Bahnhofsviertel----
                venue  freq
0               Hotel  0.17
1  Italian Restaurant  0.09
2          Restaurant  0.09
3              Bistro  0.09
4        Cocktail Bar  0.04


----Bockenheim----
                venue  freq
0                Café  0.12
1    Asian Restaurant  0.12
2  Spanish Restaurant  0.08
3      Ice Cream Shop  0.08
4         Supermarket  0.08


----Gallus----
              venue  freq
0  Asian Restaurant  0.13
1              Café  0.13
2            Museum  0.07
3    Boarding House  0.07
4               Pub  0.07


----Gutleutviertel----
                venue  freq
0               Hotel  0.17
1  Italian Restaurant  0.09
2          Restaurant  0.09
3              Bistro  0.09
4        Cocktail Bar  0.04


----Innenstadt----
                venue  freq
0  Italian Restau

#### Let's put that into a *pandas* dataframe

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [99]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = FFM_grouped['Neighborhood']

for ind in np.arange(FFM_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(FFM_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Altstadt,Café,Italian Restaurant,Restaurant,Department Store,Bar,Thai Restaurant,Plaza,Burger Joint,Coffee Shop,Nightclub
1,Bahnhofsviertel,Hotel,Restaurant,Italian Restaurant,Bistro,Korean Restaurant,Cocktail Bar,Chinese Restaurant,Seafood Restaurant,River,Mexican Restaurant
2,Bockenheim,Asian Restaurant,Café,Spanish Restaurant,Ice Cream Shop,Supermarket,Cocktail Bar,Hookah Bar,Wine Bar,Middle Eastern Restaurant,Department Store
3,Gallus,Asian Restaurant,Café,Pub,Greek Restaurant,Concert Hall,Museum,Event Space,Boarding House,Food Court,Grocery Store
4,Gutleutviertel,Hotel,Restaurant,Italian Restaurant,Bistro,Korean Restaurant,Cocktail Bar,Chinese Restaurant,Seafood Restaurant,River,Mexican Restaurant
5,Innenstadt,Italian Restaurant,Café,Hotel,Thai Restaurant,Indian Restaurant,Bar,Pizza Place,Yoga Studio,Burger Joint,Indie Movie Theater
6,Nordend,Café,Italian Restaurant,Ice Cream Shop,Plaza,Wine Bar,Pub,Bar,Bakery,Pizza Place,Vietnamese Restaurant
7,Westend,Italian Restaurant,Steakhouse,German Restaurant,French Restaurant,Bakery,Café,Supermarket,Pizza Place,Salad Place,Burrito Place
